In [11]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.10.1


In [12]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [13]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다


Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.3388 - accuracy: 0.6107
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.1879 - accuracy: 0.6540 - val_loss: 0.7651 - val_accuracy: 0.7710
Epoch 2/10
23/32 [====================>.........] - ETA: 0s - loss: 0.4437 - accuracy: 0.8845
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.4329 - accuracy: 0.8860 - val_loss: 0.5348 - val_accuracy: 0.8300
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.3030 - accuracy: 0.9279
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2965 - accuracy: 0.9310 - val_loss: 0.4759 - val_accuracy: 0.8500
Epoch 4/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2177 - accuracy: 0.9500
Epoch 4: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 11ms/step - los

In [15]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4167 - accuracy: 0.0680 - 340ms/epoch - 11ms/step
훈련되지 않은 모델의 정확도:  6.80%


In [16]:
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4298 - accuracy: 0.8700 - 140ms/epoch - 4ms/step
복원된 모델의 정확도: 87.00%


In [21]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2\cp-0005.ckpt

Epoch 10: saving model to training_2\cp-0010.ckpt

Epoch 15: saving model to training_2\cp-0015.ckpt

Epoch 20: saving model to training_2\cp-0020.ckpt

Epoch 25: saving model to training_2\cp-0025.ckpt

Epoch 30: saving model to training_2\cp-0030.ckpt

Epoch 35: saving model to training_2\cp-0035.ckpt

Epoch 40: saving model to training_2\cp-0040.ckpt

Epoch 45: saving model to training_2\cp-0045.ckpt

Epoch 50: saving model to training_2\cp-0050.ckpt


In [23]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4966 - accuracy: 0.8760 - 278ms/epoch - 9ms/step
복원된 모델의 정확도: 87.60%


In [28]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1327 - accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4039 - accuracy: 0.8890
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2807 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2013 - accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1527 - accuracy: 0.9710
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [29]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 512)               401920    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4278 - accuracy: 0.8620 - 105ms/epoch - 3ms/step
복원된 모델의 정확도: 86.20%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


In [35]:
#hd5
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1726 - accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4399 - accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2872 - accuracy: 0.9350
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2089 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1609 - accuracy: 0.9680


In [33]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 512)               401920    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [36]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4014 - accuracy: 0.8720 - 97ms/epoch - 3ms/step
복원된 모델의 정확도: 100.00%
